# Prepare dataset

In [2]:
!pip install tensorflow
import tensorflow as tf

In [3]:
# The code was removed by DSX for sharing.

,Material,Usura_mat_g,Durezza_Disco,Comprex_LP,Comprex_LR,Grindo_LP,Grindo_LR,CodPinza_0,CodPinza_1,CodPinza_3,...,PistoneCpx_54,PistoneCpx_57,Inerzia_45.0,Inerzia_51.0,Inerzia_65.0,Inerzia_72.5,Raggio_medio_104.0,Raggio_medio_113.0,Raggio_medio_117.7,Raggio_medio_118.0
0,2,15.2,165,182,182,810,813,0,1,0,...,1,0,0,0,1,0,0,0,0,1
1,2,13.3,177,181,203,738,729,0,1,0,...,1,0,0,0,1,0,0,0,0,1
2,1,7.5,235,125,136,727,731,0,1,0,...,1,0,0,0,1,0,0,0,0,1
3,1,7.8,237,137,102,701,674,0,1,0,...,1,0,0,0,1,0,0,0,0,1
4,2,10.9,180,196,201,820,805,0,1,0,...,1,0,0,0,1,0,0,0,0,1


In [4]:
# balance materials --> min max scale

In [5]:
# get one hot encoded labels
labels = pd.get_dummies(df_data_1, columns=["Material"])
labels = pd.DataFrame(labels[["Material_0", "Material_1", "Material_2", "Material_3","Material_4"]])

labels.head()

,Material_0,Material_1,Material_2,Material_3,Material_4
0,0,0,1,0,0
1,0,0,1,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0


In [6]:
# remove the class column from the entire dataset
dataset = df_data_1.drop(["Material"], axis=1)
dataset.head()

,Usura_mat_g,Durezza_Disco,Comprex_LP,Comprex_LR,Grindo_LP,Grindo_LR,CodPinza_0,CodPinza_1,CodPinza_3,CodPinza_4,...,PistoneCpx_54,PistoneCpx_57,Inerzia_45.0,Inerzia_51.0,Inerzia_65.0,Inerzia_72.5,Raggio_medio_104.0,Raggio_medio_113.0,Raggio_medio_117.7,Raggio_medio_118.0
0,15.2,165,182,182,810,813,0,1,0,0,...,1,0,0,0,1,0,0,0,0,1
1,13.3,177,181,203,738,729,0,1,0,0,...,1,0,0,0,1,0,0,0,0,1
2,7.5,235,125,136,727,731,0,1,0,0,...,1,0,0,0,1,0,0,0,0,1
3,7.8,237,137,102,701,674,0,1,0,0,...,1,0,0,0,1,0,0,0,0,1
4,10.9,180,196,201,820,805,0,1,0,0,...,1,0,0,0,1,0,0,0,0,1


In [7]:
# partition into trainset and and test set
from sklearn.cross_validation import StratifiedShuffleSplit

holdout_split_column = df_data_1["Material"] # Split data to get an equal class distribution between the test and train set
sss = StratifiedShuffleSplit(holdout_split_column, 1, test_size=.15, random_state=0)

X_train, X_test = None, None
y_train, y_test = None, None

for train_index, test_index in sss:
    X_train, X_test = dataset.values[train_index], dataset.values[test_index]
    y_train, y_test = labels.values[train_index], labels.values[test_index]
    
print "Train set size", X_train.shape[0]
print "Test set size", X_test.shape[0]

Train set size 554
Test set size 97


# Create model

In [8]:
# Model parameters

n_features = X_train.shape[1]
n_classes = y_train.shape[1]
n_samples = X_train.shape[0]
n_steps = 800

In [9]:
# Create the model

graph = tf.Graph()
with graph.as_default():
    weights = tf.Variable(tf.truncated_normal([n_features, n_classes]))
    x_tf = tf.placeholder(tf.float32, shape=(n_samples, n_features))
    biases = tf.Variable(tf.zeros([n_classes]))
    logits = tf.matmul(x_tf, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_train, logits=logits))

    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
#     test_prediction = tf.nn.softmax(tf.matmul(X_test, weights) + biases)

In [10]:
# Run the model

with tf.Session(graph=graph) as session: 
    tf.initialize_all_variables()
    
    for step in range(n_steps):
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict={x_tf: X_train})
    if (step % 100 == 0):
        print('Loss at step %d: %f' % (step, l))
        print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
        print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))



FailedPreconditionError: Attempting to use uninitialized value Variable_1
	 [[Node: Variable_1/read = Identity[T=DT_FLOAT, _class=["loc:@Variable_1"], _device="/job:localhost/replica:0/task:0/cpu:0"](Variable_1)]]
Caused by op u'Variable_1/read', defined at:
  File "/usr/local/src/bluemix_jupyter_bundle.v43/notebook/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/local/src/bluemix_jupyter_bundle.v43/notebook/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/src/bluemix_jupyter_bundle.v43/notebook/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/src/bluemix_jupyter_bundle.v43/notebook/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/usr/local/src/bluemix_jupyter_bundle.v43/notebook/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 442,